<a href="https://colab.research.google.com/github/Zel322/Kapeng-Barako-Mask-RCNN/blob/main/Colab_Notebook_barako_maskrcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mask R-CNN instance segmentation with Kapeng Barako dataset in Google Colab
Jupyter notebook providing steps to train a **Matterport Mask R-CNN** model with custom dataset.

It runs in [Google Colab](https://colab.research.google.com/) using [Matterport framework](https://github.com/matterport/Mask_RCNN) with TensorFlow backend.

**Requirements are only dataset images and annotations file.**

**Colab Runtime type: Python3, GPU enabled.**

#Install miniconda and python 3.6

In [ ]:
%env PYTHONPATH=


In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX



In [ ]:
!python --version

#Clone Mask RCNN Repository

In [ ]:
%cd
  
!git clone --quiet https://github.com/matterport/Mask_RCNN.git

#Install required packages


In [ ]:
#run when tensorflow is not found

!pip install tensorflow-gpu==1.5
!pip install keras==2.1.6

In [ ]:
%cd ~/Mask_RCNN

!pip install -q PyDrive
!pip install --upgrade pip
#!pip install -r requirements.txt
!python setup.py install



In [ ]:
!pip install numpy
!pip install -U scikit-image==0.16.2
!pip install opencv-python
!pip install ipython
!pip install ipykernel
!pip install h5py==2.10.0
#python --version

In [ ]:
!pip install pandas
!pip install seaborn
!pip install sklearn

#Change CUDA Version from 10 to 9


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

In [ ]:
!nvcc --version

#Download and extract dataset and other uploaded files 
Update fileId variable with Google Drive id of your image.zip dataset and upload.zip files



In [ ]:
%cd /root/Mask_RCNN

In [ ]:
methoimport os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#fileId = '1FlT3MFK4VFwTcn575FlKpqs65dyH6E7Z'#dataset without overripe
fileId = '1pfVNJoSP0Vpqwv5dxIQo2ibfd7Jv_Wmz'#dataset without overripe

os.makedirs('dataset', exist_ok=True)
os.chdir('dataset')

fileName = fileId + '.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

In [ ]:
%cd ~/Mask_RCNN

In [ ]:
fileId2 = '1G0A3I1VCN2CatZ9tmywn2XHDmjpaEDDr'#for_5.zip from institutional account

os.makedirs('uploads', exist_ok=True)
os.chdir('uploads')

fileName2 = fileId2 + '.zip'
downloaded = drive.CreateFile({'id': fileId2})
downloaded.GetContentFile(fileName2)
ds = ZipFile(fileName2)
ds.extractall()
os.remove(fileName2)
print('Extracted zip file ' + fileName2)

#Install cudnn 7.0.5

the .deb file is uploaded to the drive

In [ ]:
!dpkg -i "/root/Mask_RCNN/uploads/libcudnn7_7.0.5.15-1+cuda9.0_amd64.deb"

#Edit settings file
*  modified python file is uploaded to the drive
*  set epochs number


In [ ]:
%cd ~/Mask_RCNN/uploads

!sed -n -e '/epochs/p' barako_v2.py
!sed -n -e '/STEPS_PER_EPOCH/p' barako_v2.py
!sed -n -e '/DETECTION_MIN_CONFIDENCE/p' barako_v2.py
!sed -n -e '/calculate_map_at_every_X_epoch/p' barako_v2.py
!sed -n -e '/target_mAP/p' barako_v2.py


In [ ]:
!sed -i -- 's/epochs=10/epochs=50/g' barako_v2.py
!sed -i -- 's/STEPS_PER_EPOCH = 100/STEPS_PER_EPOCH = 175/g' barako_v2.py
!sed -i -- 's/DETECTION_MIN_CONFIDENCE = 0.9/DETECTION_MIN_CONFIDENCE = 0.7/g' barako_v2.py
!sed -i -- 's/calculate_map_at_every_X_epoch=10/calculate_map_at_every_X_epoch=35/g' barako_v2.py
!sed -i -- 's/target_mAP=0.8/target_mAP=0.6/g' barako_v2.py

#Train model
Pretrained weights options are COCO, ImageNet or a model trained before


In [ ]:
%cd ~/Mask_RCNN/uploads

!python barako_v2.py train --dataset=/root/Mask_RCNN/dataset/ --weights=coco #model.py modifications: set augment and augmentation to True

In [ ]:

%cd ~/Mask_RCNN/uploads

from google.colab import files
#import modified_model as model
files.download("/root/logs/barako20210321T0828/mask_rcnn_barako_0060.h5")

/root/Mask_RCNN


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Run inference on test dataset

move the following files to MaskRCNN


*   modified_utils.py
*   modified_model.py
*   modified_visualize.py
*   barako_v2.py
*   barako_inference.py
*   modified_model.py

*due to compatibility issues with the tensorflow versions in Colab, the prediction images will not be present. A jupyter notebook (inspect_model) from the MaskRCNN github can be used to visualize the results. A webpage User Interface will also be uploaded in the future.




In [ ]:
%cd ~/Mask_RCNN 
import os
os.makedirs('predictions', exist_ok=True)
!python barako_inference.py 